In [1]:
!pip install language-tool-python

In [ ]:
homework_text = """
Describe an event which you did not look forward to attending but

turned out to exciting.

As a young teenager growing up in Singapore, I often found myself dreading certain
events or activities that my fameely and friends wood plan. Whether it was a family
gathering, a school field trip, or a friend's birthday party, there were times when I just
did not feel like going. However, one particular event standed out in my memorie as an
occasion where I surprised myslef by having an unexpectedly great time.

It was a warmed Saturdays afternoon, and my mom has told me that we were going to
visit my grate aunt who lived in a different part of the city. I was not particularly close
to my great aunt, and I did not really know what to expect from the visit. In fact, I was
feeling a bit grahmpy and resentful about having to spend my weekend doing
something that I was not interesteded in. As we made our way to her house, I kept
thinking about all the other things I could have been doing instead.

When we arrived, my great aunt greeted us with a warm smile and a big hug. She
was a kind and gentle woman, and I immediately felt at ease in her company. As we
settled into her living room, she offer us some snacks and began to tell us stories
about her life. I was surprised to find that I was actually interest in what she had to
say, and I listened attentively as she shared her experiences growing up in
Singapore during a different era.

After a while, my greatest of aunt suggested that we take a walks around her
neighbourhood. I wasn't not sure what to expect, but I agreed to join her and my mom.
As we walked, my great aunt pointed out different landmarks and shared more
stories about her past. I found myself enjoying the walk, taking in the sights and

sounds of the neighborhood, and feeling grateful to be spending time with my family.
"""

In [25]:
from spellchecker import SpellChecker
import json

def spelling_only_json(text):
    cleaned_text = text.replace('.', '').replace(',', '').replace(';', '').replace(':', '')
    words = cleaned_text.split()
    
    spell = SpellChecker()
    misspelled = spell.unknown(words)
    
    result = {
        "Misspelled_Count": len(misspelled),
        "Misspelled_Words": list(misspelled)
    }
    
    return result

result = spelling_only_json(homework_text)
print(json.dumps(result, indent=2))


{
  "Misspelled_Count": 7,
  "Misspelled_Words": [
    "neighbourhood",
    "myslef",
    "interesteded",
    "memorie",
    "standed",
    "grahmpy",
    "fameely"
  ]
}


In [30]:
import spacy
import json

nlp = spacy.load("en_core_web_sm")

def grammar_checker_spacy_json(text):
    """
    Returns a JSON-like dictionary with:
    - Number of detected grammar issues
    - List of issues with messages and sentence context
    """
    doc = nlp(text)
    errors = []

    for sent in doc.sents:
        # 1. Run-on / long sentences
        if len(sent) > 40:  # arbitrary threshold
            errors.append({
                "Message": "Long sentence / possible run-on sentence",
                "Context": sent.text
            })
        # 2. Repeated words
        words = [token.text.lower() for token in sent if token.is_alpha]
        for i in range(len(words)-1):
            if words[i] == words[i+1]:
                errors.append({
                    "Message": f"Repeated word: '{words[i]}'",
                    "Context": sent.text
                })
        # 3. Simple subject-verb agreement check (heuristic)
        for token in sent:
            if token.dep_ == "nsubj":
                verb = [child for child in token.head.children if child.dep_ == "aux" or child.pos_ == "VERB"]
                if verb:
                    # basic check: singular/plural mismatch
                    if token.tag_ in ["NN", "NNP"] and token.head.tag_ == "VBP":
                        errors.append({
                            "Message": f"Possible subject-verb disagreement with '{token.text} {token.head.text}'",
                            "Context": sent.text
                        })

    result = {
        "Grammar_Error_Count": len(errors),
        "Grammar_Errors": errors
    }
    return result

# Example usage
essay_text = "I was going to the store yesterday. He go to school every day. This is is a long sentence that goes on and on without proper punctuation or separation making it very hard to read."
result = grammar_checker_spacy_json(essay_text)
print(json.dumps(result, indent=2))


{
  "Grammar_Error_Count": 1,
  "Grammar_Errors": [
    {
      "Message": "Repeated word: 'is'",
      "Context": "This is is a long sentence that goes on and on without proper punctuation or separation making it very hard to read."
    }
  ]
}
